# Particle Filter for Crazyflie: Fault-Tolerant State Estimation

This notebook demonstrates **Particle Filter (PF)** for robust state estimation with **intermittent GPS dropouts**.

**Problem**: Maintain accurate state estimate during sensor failures

**State**: $[x, y, z, v_x, v_y, v_z]$ (3D position and velocity)

**Measurements**: 
- GPS (intermittent - simulated dropouts)
- IMU (continuous)

**Challenge**: Particle filter maintains multi-modal belief during GPS outages, while KF/UKF would diverge or lose track.

**Reference**: Gordon et al. (1993), "Novel approach to nonlinear/non-Gaussian Bayesian state estimation"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from pykal import DynamicalSystem, ROSNode
from pykal.algorithm_library.estimators.pf import PF
from pykal.gazebo import start_gazebo, stop_gazebo

np.random.seed(42)

## 1. Launch Gazebo Simulation

Start Crazyflie at 1m altitude.

In [ ]:
from pykal.gazebo import start_gazebo, stop_gazebo
import time

gz = start_gazebo(
    robot='crazyflie',
    world='empty',
    headless=True
)

print("Gazebo launched successfully!")
time.sleep(2)


## 2. Fault-Tolerant Estimation Setup

### Dynamics Model
Constant velocity with IMU-based acceleration:
$$\begin{bmatrix} x \\ y \\ z \\ v_x \\ v_y \\ v_z \end{bmatrix}_k = \begin{bmatrix} x + v_x \Delta t \\ y + v_y \Delta t \\ z + v_z \Delta t \\ v_x + a_x \Delta t \\ v_y + a_y \Delta t \\ v_z + a_z \Delta t \end{bmatrix}_{k-1} + \mathbf{w}_k$$

### Measurement Models
**GPS** (when available):
$$\mathbf{z}_{GPS} = [x, y, z] + \mathbf{v}_{GPS}$$

**IMU** (always available):
$$\mathbf{z}_{IMU} = [a_x, a_y, a_z] + \mathbf{v}_{IMU}$$

In [ ]:
# Particle filter parameters
N_PARTICLES = 500
dt = 0.1  # 10 Hz

# Process noise (higher during GPS dropout)
PROCESS_NOISE_GPS = np.diag([0.01, 0.01, 0.01, 0.05, 0.05, 0.05])
PROCESS_NOISE_NO_GPS = np.diag([0.05, 0.05, 0.05, 0.2, 0.2, 0.2])  # Higher uncertainty

# Measurement noise
GPS_NOISE = 0.5  # GPS accuracy (m)
IMU_NOISE = 0.1  # IMU accelerometer noise (m/s²)

# GPS dropout simulation
GPS_DROPOUT_PROB = 0.3  # 30% chance of dropout
GPS_DROPOUT_DURATION = 2.0  # 2 second outages

print(f"Particle Filter Parameters:")
print(f"  Particles: {N_PARTICLES}")
print(f"  GPS noise: {GPS_NOISE}m")
print(f"  GPS dropout probability: {GPS_DROPOUT_PROB*100}%")
print(f"  GPS dropout duration: {GPS_DROPOUT_DURATION}s")

## 3. Initialize Particles

In [ ]:
# Initial state estimate: [x, y, z, vx, vy, vz]
x0 = np.array([0.0, 0.0, 1.0, 0.0, 0.0, 0.0])
P0 = np.diag([0.5, 0.5, 0.2, 0.1, 0.1, 0.1])

# Initialize particles
particles_init = np.random.multivariate_normal(x0, P0, N_PARTICLES)
weights_init = np.ones(N_PARTICLES) / N_PARTICLES

print(f"Initialized {N_PARTICLES} particles")
print(f"Initial position: {x0[:3]}")
print(f"Initial velocity: {x0[3:]}")

## 4. Dynamics and Measurement Functions

In [ ]:
def dynamics_with_imu(state, imu_accel, has_gps):
    """
    State propagation using IMU measurements.
    
    Parameters
    ----------
    state : np.ndarray
        Current state [x, y, z, vx, vy, vz]
    imu_accel : np.ndarray
        IMU acceleration [ax, ay, az]
    has_gps : bool
        Whether GPS is available (affects process noise)
    
    Returns
    -------
    np.ndarray
        Propagated state
    """
    x, y, z, vx, vy, vz = state
    ax, ay, az = imu_accel
    
    # Constant velocity + IMU acceleration
    x_new = x + vx * dt
    y_new = y + vy * dt
    z_new = z + vz * dt
    vx_new = vx + ax * dt
    vy_new = vy + ay * dt
    vz_new = vz + (az - 9.81) * dt  # Remove gravity
    
    state_new = np.array([x_new, y_new, z_new, vx_new, vy_new, vz_new])
    
    # Add process noise (higher without GPS)
    if has_gps:
        noise = np.random.multivariate_normal(np.zeros(6), PROCESS_NOISE_GPS)
    else:
        noise = np.random.multivariate_normal(np.zeros(6), PROCESS_NOISE_NO_GPS)
    
    return state_new + noise

def gps_likelihood(gps_meas, state):
    """
    GPS measurement likelihood.
    
    Parameters
    ----------
    gps_meas : np.ndarray
        GPS position [x, y, z]
    state : np.ndarray
        Particle state [x, y, z, vx, vy, vz]
    
    Returns
    -------
    float
        Likelihood value
    """
    # Extract position from state
    pos_predicted = state[:3]
    
    # Gaussian likelihood
    diff = gps_meas - pos_predicted
    likelihood = np.exp(-0.5 * np.sum(diff**2) / GPS_NOISE**2)
    
    return likelihood + 1e-10  # Avoid zero

print("Dynamics and measurement functions defined")

## 5. GPS Dropout Simulator

In [ ]:
class GPSDropoutSimulator:
    """
    Simulates intermittent GPS dropouts.
    """
    def __init__(self, dropout_prob, dropout_duration):
        self.dropout_prob = dropout_prob
        self.dropout_duration = dropout_duration
        self.in_dropout = False
        self.dropout_start_time = 0.0
    
    def is_gps_available(self, current_time):
        """
        Check if GPS is available at current time.
        """
        if self.in_dropout:
            # Check if dropout period is over
            if current_time - self.dropout_start_time >= self.dropout_duration:
                self.in_dropout = False
                print(f"  GPS RESTORED at t={current_time:.1f}s")
                return True
            else:
                return False
        else:
            # Random dropout
            if np.random.random() < self.dropout_prob * 0.1:  # Per-step probability
                self.in_dropout = True
                self.dropout_start_time = current_time
                print(f"  GPS DROPOUT at t={current_time:.1f}s")
                return False
            else:
                return True

gps_simulator = GPSDropoutSimulator(GPS_DROPOUT_PROB, GPS_DROPOUT_DURATION)
print("GPS dropout simulator initialized")

## 6. Particle Filter ROS Node

In [ ]:
from geometry_msgs.msg import PoseStamped, Vector3Stamped
from sensor_msgs.msg import Imu

# Global state
particles_current = particles_init.copy()
weights_current = weights_init.copy()
state_history = {'time': [], 'state': [], 'gps_available': [], 'particle_spread': []}

def pf_callback(tk, gps_pose, imu_accel):
    """
    Particle filter callback with GPS dropout handling.
    
    Parameters
    ----------
    tk : float
        Current time
    gps_pose : np.ndarray
        GPS position [x, y, z]
    imu_accel : np.ndarray
        IMU acceleration [ax, ay, az]
    
    Returns
    -------
    dict
        Estimated state
    """
    global particles_current, weights_current
    
    # Check GPS availability
    has_gps = gps_simulator.is_gps_available(tk)
    
    # Prediction step (using IMU)
    for i in range(N_PARTICLES):
        particles_current[i] = dynamics_with_imu(
            particles_current[i], 
            imu_accel, 
            has_gps
        )
    
    # Update step (only if GPS available)
    if has_gps:
        # Weight update with GPS
        for i in range(N_PARTICLES):
            weights_current[i] = gps_likelihood(gps_pose, particles_current[i])
        
        # Normalize weights
        weights_current /= (weights_current.sum() + 1e-10)
        
        # Resample if needed
        N_eff = 1.0 / (np.sum(weights_current**2) + 1e-10)
        if N_eff < N_PARTICLES / 2:
            # Systematic resampling
            cumsum = np.cumsum(weights_current)
            positions = (np.arange(N_PARTICLES) + np.random.random()) / N_PARTICLES
            indices = np.searchsorted(cumsum, positions)
            particles_current = particles_current[indices]
            weights_current = np.ones(N_PARTICLES) / N_PARTICLES
    
    # State estimate (weighted mean)
    state_est = np.average(particles_current, axis=0, weights=weights_current)
    
    # Compute particle spread (uncertainty measure)
    particle_spread = np.std(particles_current[:, :3], axis=0).mean()
    
    # Log history
    state_history['time'].append(tk)
    state_history['state'].append(state_est.copy())
    state_history['gps_available'].append(has_gps)
    state_history['particle_spread'].append(particle_spread)
    
    return {'state_estimate': state_est}

print("Particle filter callback defined")

## 7. Create and Run PF Node

In [ ]:
from std_msgs.msg import Float64MultiArray

# Create PF node
pf_node = ROSNode(
    callback=pf_callback,
    subscribes_to=[
        ('/crazyflie/gps/pose', PoseStamped, 'gps_pose'),
        ('/crazyflie/imu/accel', Vector3Stamped, 'imu_accel')
    ],
    publishes_to=[
        ('state_estimate', Float64MultiArray, '/crazyflie/pf/state')
    ],
    node_name='crazyflie_pf',
    rate_hz=10.0
)

print("PF node created")
print("\nSubscriptions:")
print("  - /crazyflie/gps/pose")
print("  - /crazyflie/imu/accel")
print("\nPublications:")
print("  - /crazyflie/pf/state")

In [ ]:
# Start PF node
pf_node.create_node()
pf_node.start()

print("\nParticle Filter node running!")
print("Monitoring GPS dropouts and state estimation...\n")

# Run for 30 seconds
try:
    time.sleep(30)
except KeyboardInterrupt:
    print("\nInterrupted by user")

# Stop node
pf_node.stop()
print("\nPF node stopped")

## 8. Analyze Results

In [ ]:
if len(state_history['time']) > 0:
    time_arr = np.array(state_history['time'])
    state_arr = np.array(state_history['state'])
    gps_avail = np.array(state_history['gps_available'])
    spread_arr = np.array(state_history['particle_spread'])
    
    # Identify dropout periods
    dropout_periods = []
    in_dropout = False
    start_idx = 0
    
    for i in range(len(gps_avail)):
        if not gps_avail[i] and not in_dropout:
            in_dropout = True
            start_idx = i
        elif gps_avail[i] and in_dropout:
            in_dropout = False
            dropout_periods.append((start_idx, i))
    
    print(f"\n=== GPS Dropout Analysis ===")
    print(f"Total dropouts: {len(dropout_periods)}")
    for idx, (start, end) in enumerate(dropout_periods):
        duration = time_arr[end] - time_arr[start]
        spread_increase = spread_arr[end] - spread_arr[start]
        print(f"  Dropout {idx+1}: {duration:.2f}s, spread increase: {spread_increase:.3f}m")
else:
    print("No data collected")

## 9. Visualize Results

In [ ]:
if len(state_history['time']) > 0:
    fig, axes = plt.subplots(3, 2, figsize=(14, 12))
    
    # Position estimates
    axes[0, 0].plot(time_arr, state_arr[:, 0], 'b-', linewidth=2, label='X position')
    axes[0, 0].plot(time_arr, state_arr[:, 1], 'r-', linewidth=2, label='Y position')
    axes[0, 0].plot(time_arr, state_arr[:, 2], 'g-', linewidth=2, label='Z position')
    
    # Highlight dropout periods
    for start, end in dropout_periods:
        axes[0, 0].axvspan(time_arr[start], time_arr[end], alpha=0.3, color='red')
    
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('Position (m)')
    axes[0, 0].set_title('Position Estimates', fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Velocity estimates
    axes[0, 1].plot(time_arr, state_arr[:, 3], 'b-', linewidth=2, label='vx')
    axes[0, 1].plot(time_arr, state_arr[:, 4], 'r-', linewidth=2, label='vy')
    axes[0, 1].plot(time_arr, state_arr[:, 5], 'g-', linewidth=2, label='vz')
    
    for start, end in dropout_periods:
        axes[0, 1].axvspan(time_arr[start], time_arr[end], alpha=0.3, color='red')
    
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Velocity (m/s)')
    axes[0, 1].set_title('Velocity Estimates', fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Particle spread (uncertainty)
    axes[1, 0].plot(time_arr, spread_arr, 'purple', linewidth=2)
    
    for start, end in dropout_periods:
        axes[1, 0].axvspan(time_arr[start], time_arr[end], alpha=0.3, color='red', label='GPS dropout' if start == dropout_periods[0][0] else '')
    
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Particle Spread (m)')
    axes[1, 0].set_title('Uncertainty Evolution', fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # GPS availability
    axes[1, 1].plot(time_arr, gps_avail.astype(int), 'ko-', markersize=3, linewidth=1)
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('GPS Available')
    axes[1, 1].set_title('GPS Availability', fontweight='bold')
    axes[1, 1].set_ylim([-0.1, 1.1])
    axes[1, 1].set_yticks([0, 1])
    axes[1, 1].set_yticklabels(['Dropout', 'Available'])
    axes[1, 1].grid(True)
    
    # 3D trajectory
    from mpl_toolkits.mplot3d import Axes3D
    ax_3d = fig.add_subplot(3, 2, 5, projection='3d')
    ax_3d.plot(state_arr[:, 0], state_arr[:, 1], state_arr[:, 2], 'b-', linewidth=2)
    ax_3d.scatter(state_arr[0, 0], state_arr[0, 1], state_arr[0, 2], c='g', s=100, marker='o', label='Start')
    ax_3d.scatter(state_arr[-1, 0], state_arr[-1, 1], state_arr[-1, 2], c='r', s=100, marker='*', label='End')
    ax_3d.set_xlabel('X (m)')
    ax_3d.set_ylabel('Y (m)')
    ax_3d.set_zlabel('Z (m)')
    ax_3d.set_title('3D Trajectory', fontweight='bold')
    ax_3d.legend()
    
    # Final particle distribution (2D projection)
    axes[2, 1].scatter(particles_current[:, 0], particles_current[:, 1],
                      s=weights_current * 1000, alpha=0.3, c='blue')
    axes[2, 1].plot(state_arr[-1, 0], state_arr[-1, 1], 'r*', markersize=20, label='Final estimate')
    axes[2, 1].set_xlabel('X position (m)')
    axes[2, 1].set_ylabel('Y position (m)')
    axes[2, 1].set_title('Final Particle Distribution (XY)', fontweight='bold')
    axes[2, 1].legend()
    axes[2, 1].grid(True)
    axes[2, 1].set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey observations:")
    print("- Particle spread INCREASES during GPS dropouts (red regions)")
    print("- Particle spread DECREASES when GPS returns (converges)")
    print("- PF maintains reasonable estimates using IMU during dropouts")
else:
    print("No data to visualize")

## 10. Shutdown Gazebo

In [ ]:
stop_gazebo(gz)
print("Gazebo simulation stopped")

## Conclusion

### Key Takeaways

1. **Fault Tolerance**: Particle filter gracefully handles GPS dropouts by maintaining multi-modal belief
2. **Uncertainty Awareness**: Particle spread increases during outages, decreases on GPS return
3. **IMU Integration**: Continuous IMU measurements provide prediction during GPS absence

### PF vs KF/UKF for Sensor Failures

| Method | GPS Dropout Handling | Computational Cost |
|--------|---------------------|-------------------|
| **Particle Filter** | Maintains multi-modal belief, robust | High (N particles) |
| **KF/UKF** | Diverges or loses track | Low (single hypothesis) |
| **EKF** | Linearization errors compound | Low |

### Practical Applications

- **Urban canyon navigation**: GPS multipath/blockage
- **Indoor-outdoor transitions**: Intermittent GPS
- **Sensor fusion**: Combining unreliable sensors
- **Fault-tolerant systems**: Critical applications requiring robustness

### Performance Metrics

- **Particle spread**: Direct measure of uncertainty
- **Effective sample size (ESS)**: When to resample
- **Position error**: Compare PF vs ground truth (if available)

### Next Steps

- Add visual odometry as backup sensor
- Implement adaptive particle count (AMCL-style)
- Compare with complementary filter
- Test on real Crazyflie hardware

### Reference

Gordon, N. J., Salmond, D. J., & Smith, A. F. M. (1993). *Novel approach to nonlinear/non-Gaussian Bayesian state estimation*. IEE Proceedings F (Radar and Signal Processing), 140(2), 107-113.